* Daniel Enrique Patiño Jaimes 000421816
* David Santiago Velasco Daza 000430882



In [1]:
pip install wfdb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 161 kB 5.1 MB/s 


Importaciones necesarias

In [25]:
import wfdb
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import statistics as stat
from scipy import stats as s
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

1. Realizar un algoritmo que me permita identificar el latido cardiaco en una señal ECG utilizando patrones (características obtenidas de segmentos de la señal) utilizando como método o camino para ello redes neuronales artificiales (ANN).

Analizar la base de datos



In [24]:
db = os.listdir('/content/database')
db.sort()
data=[]
signal=[]
for i in db:
  if i.__contains__(".atr"):
    data.append(i.replace(".atr",""))

print(data)

['16265', '16272', '16273', '16420', '16483', '16539', '16773', '16786', '16795', '17052', '17453', '18177', '18184', '19088', '19090', '19093', '19140', '19830']


Extraer de las señales de cada registro

In [26]:
for i in data:
  record = wfdb.rdrecord('database/'+i)
  signal.append(record.p_signal[:,0])
  
print(signal)

[array([-0.165, -0.155, -0.195, ..., -0.05 , -0.05 , -0.05 ]), array([-0.185, -0.215, -0.215, ..., -0.04 , -0.04 , -0.04 ]), array([-0.305, -0.265, -0.245, ...,  0.01 ,  0.01 ,  0.01 ]), array([-0.095, -0.085, -0.085, ..., -0.07 , -0.07 , -0.07 ]), array([-0.075, -0.165, -0.225, ..., -0.05 , -0.06 , -0.06 ]), array([-0.185, -0.155, -0.145, ..., -0.05 , -0.05 , -0.05 ]), array([-0.355, -0.355, -0.345, ..., -0.04 , -0.04 , -0.04 ]), array([-0.275, -0.245, -0.285, ..., -0.03 , -0.03 , -0.04 ]), array([ 0.125,  0.005,  0.025, ..., -0.03 , -0.03 , -0.03 ]), array([-0.035, -0.015, -0.045, ..., -0.04 , -0.04 , -0.04 ]), array([ 0.525,  0.695,  0.475, ..., -0.02 , -0.02 , -0.02 ]), array([-0.215, -0.195, -0.185, ..., -0.05 , -0.05 , -0.05 ]), array([-0.325, -0.395, -0.255, ..., -0.11 , -0.11 , -0.1  ]), array([ 0.365,  0.365,  0.355, ..., -0.04 , -0.04 , -0.04 ]), array([ 0.165,  0.155,  0.155, ..., -0.05 , -0.05 , -0.05 ]), array([ 0.675,  0.665,  0.655, ..., -0.09 , -0.1  , -0.09 ]), array([

Obtener posición de latidos y cantidad de latidos por registro

In [ ]:
for i in data:
  annotation = wfdb.rdann('database/'+i,'atr').sample
  print(annotation) #Posiciones de latidos
  print(len(annotation)) #Cantidad de latidos

Agregar al dataset los registros con latidos y las características evaluadas (media, mediana, moda, desviación estándar y courtosis)

In [27]:
dataset = []
for i in data:
  annotation = wfdb.rdann('database/'+i,'atr').sample
  record = wfdb.rdrecord('database/'+i)
  signal = record.p_signal[:,0]
  cont = 0
  for j in annotation:
    if j > 8:
      if cont < 12000:
        frame = []
        Vobser = signal[j-7:j+7]
        media = (np.mean(Vobser))
        frame.append(media)
        moda = float(s.mode(Vobser)[0])
        frame.append(moda)
        mediana = (np.median(Vobser))
        frame.append(mediana)
        dev = (stat.pstdev(Vobser))
        frame.append(dev)
        kurt = (s.kurtosis(Vobser))
        frame.append(kurt)
        frame.append((1))
        dataset.append(frame)
      cont = cont + 1

Método de búsqueda binaria para hallar la posición de las señales que no son latidos

In [28]:
def busqueda_binaria(lista, valor):
    primero = 0
    ultimo = len(lista) - 1
    encontrado = False

    while primero <= ultimo and not encontrado:
        mitad = (primero + ultimo) // 2

        if lista[mitad] == valor:
            encontrado = True
        else:
            if valor < lista[mitad]:
                ultimo = mitad - 1
            else:
                primero = mitad + 1

    return encontrado

Agregar al dataset los registros sin latidos

In [29]:
for i in data:
  annotation = wfdb.rdann('database/'+i,'atr').sample
  record = wfdb.rdrecord('database/'+i)
  signal = record.p_signal[:,0]
  cont = 0
  for j in range(1,len(signal)+1):
    if j>8 and not(busqueda_binaria(annotation,j)):
      frame = []
      Vobser = signal[j-7:j+7]
      media = (np.mean(Vobser))
      frame.append(media)
      moda = float(s.mode(Vobser)[0])
      frame.append(moda)
      mediana = (np.median(Vobser))
      frame.append(mediana)
      dev = (stat.pstdev(Vobser))
      frame.append(dev)
      kurt = (s.kurtosis(Vobser))
      frame.append(kurt)
      frame.append(0)
      dataset.append(frame)
      cont = cont + 1
      if cont >= 12000:
        break
    

16265
16272
16273
16420
16483
16539
16773
16786
16795
17052
17453
18177
18184
19088
19090
19093
19140
19830


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Generar el dataframe


In [30]:
indices = ['Registro {}'.format(i) for i in range(1,len(dataset) + 1)]

df = pd.DataFrame(data = dataset, index = indices)

df.columns = ['Media','Moda','Mediana','Desviación','Courtosis','Latido']

df

,Media,Moda,Mediana,Desviación,Courtosis,Latido
Registro 1,0.475714,-0.165,-0.165,1.205366,-0.808752,1
Registro 2,0.490714,-0.175,-0.175,1.235728,-0.603481,1
Registro 3,0.526429,-0.155,-0.155,1.228820,-0.788607,1
Registro 4,0.467143,-0.205,-0.190,1.218495,-0.675902,1
Registro 5,0.397857,-0.905,-0.210,1.163599,-0.766079,1
...,...,...,...,...,...,...
Registro 431996,-0.219286,-0.215,-0.215,0.032008,-0.592975,0
Registro 431997,-0.212857,-0.215,-0.210,0.035087,-0.693336,0
Registro 431998,-0.202143,-0.215,-0.205,0.037307,-0.408115,0
Registro 431999,-0.190000,-0.215,-0.200,0.045000,0.144621,0


Entrenar modelo

In [31]:
df_resultado = df[['Latido']]
df_caracteristica = df[['Media','Moda','Mediana','Desviación','Courtosis']]

x_train,x_test,y_train,y_test = train_test_split(df_caracteristica,df_resultado, test_size = 0.2) #test_size = porcentaje de datos para testear

clf = MLPClassifier(max_iter=200).fit(x_train,y_train)
clf.score(x_test,y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


0.9416435185185185

**Conclusiones**
*    Al trabajar con una cantidad de datos tan grande, los tiempos de ejecución son bastante altos si no se busca una manera de optimizar el trabajo.
*    Se tuvo que realizar la implementación del método de búsqueda binario para identificar las posiciones que no tienen latido obteniendo una disminución bastante grande de tiempo.
*    Una correcta elección de las características puede facilitar el trabajo para el entrenamiento de la red neuronal.
*    Se obtuvo un desempeño aproximado de 93%
*    Las pruebas realizadas son los bloques de código en los que se extraen las señales y las anotaciones, después de identificar la manera de extraerlo se implementó el código con cantidades bajas de datos para comprobar su funcionamiento y posterior a eso se usó la cantidad completa de información.

2. Con las características de configuración de la red neuronal formada en el anterior segmento.
Aplicar dicha red sobre una ventana deslizante, graficar la salida de esta red y compararla con la señal de ECG.

Predicción de la red neuronal
*    Gráfica azul es ECG.
*    Gráfica naranja es la predicción.

In [ ]:
def recorrer(index):
  i = index
  iini = (i*14)-13
  ifin = (i*14)
  return i, iini, ifin

fig = plt.figure(figsize=(20,20))
fig.tight_layout()
cont = 0
for j in data:
  cont = cont+1
  diagramaResultado = []
  diagramaEsperado = []
  record = wfdb.rdrecord('database/'+j)
  signal = record.p_signal[:,0]
  for i in range (1,432001): #Recomendación: Definir el rango que se quiere graficar para obtener resultados más rápido
    index,posi,posf = recorrer(i)
    frame = []
    Vobser = signal[posi:posf]
    media = (np.mean(Vobser))
    frame.append(media)
    moda = float(s.mode(Vobser)[0])
    frame.append(moda)
    mediana = (np.median(Vobser))
    frame.append(mediana)
    dev = (stat.pstdev(Vobser))
    frame.append(dev)
    kurt = (s.kurtosis(Vobser))
    frame.append(kurt)
    frame = [frame] #Caracteríticas para hacer la predicción
    diagramaResultado.append(clf.predict(frame))
    diagramaEsperado.append(media)
  ax = plt.subplot(6,3,cont)
  ax.plot(diagramaEsperado[0:200]) #Señal de la base de datos
  ax.plot(diagramaResultado[0:200]) #Señal obtenida por la red neuronal
  ax.set_title('Registro: '+str(j))


Se truncaron las últimas líneas 5000 del resultado de transmisión.
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but MLPClas

**¿Qué problemas puedes observar usando únicamente la gráfica?**

*   Las frecuencias muy bajas son difíciles de predecir.
*   Cuando la señal no sufre de cambios drásticos los resultados pueden no tener coherencia (Registros 19088,19090,19093,19140,19830).





**¿Por qué se puede deber eso?**
*    Porque al entrenar la red neuronal con datos numéricos, un registro con frecuencias bajas puede llegarse a confundir a un resultado sin latido.
*    Los resultados sin coherencia pueden ser producto del tamaño de la ventana deslizante.

**Mencione una estrategia que puedas utilizar o implementar para mejorarlo.**
*    Agregar más características que permitan a la red neuronal identificar de mejor manera las bajas frecuencias.
*    Realizar pruebas y cambiar el tamaño de la ventana deslizante para identificar cambios positivos al ampliar o disminuir el tamaño.